In [19]:
from collections import defaultdict

from workflow.external.gtdb_metadata import GtdbMetadataR207
from workflow.tree.pplacer_non_reps_at_pct_arc_get_taxonomy import PplacerNonRepsAtPctArcGetTaxonomy
from workflow.tree.pplacer_non_reps_at_pct_bac_get_taxonomy import PplacerNonRepsAtPctBacGetTaxonomy
import pandas as pd

In [20]:

DF_META = GtdbMetadataR207().output().read_cached()

In [21]:
def load_changes():
    out = defaultdict(dict)
    df_bac = PplacerNonRepsAtPctBacGetTaxonomy().output().read_cached()
    df_arc = PplacerNonRepsAtPctArcGetTaxonomy().output().read_cached()
    df_merged = pd.concat([df_bac, df_arc], ignore_index=True)

    for row in df_merged.itertuples():
        out[row.gid][row.pct] = row.tax
    return out

CHANGES = load_changes()
CHANGES['GCA_001896735.1']

{10: 'd__Bacteria;p__Spirochaetota;c__Spirochaetia;o__Sphaerochaetales;f__Sphaerochaetaceae;g__Sphaerochaeta;s__Sphaerochaeta sp002432715',
 15: 'd__Bacteria;p__Spirochaetota;c__Spirochaetia;o__Sphaerochaetales;f__Sphaerochaetaceae;g__Sphaerochaeta;s__Sphaerochaeta sp002432715',
 30: 'd__Bacteria;p__Spirochaetota;c__Spirochaetia;o__Sphaerochaetales;f__Sphaerochaetaceae;g__Sphaerochaeta;s__Sphaerochaeta sp002432715',
 40: 'd__Bacteria;p__Spirochaetota;c__Spirochaetia;o__Sphaerochaetales;f__Sphaerochaetaceae;g__Sphaerochaeta;s__Sphaerochaeta sp002432715',
 50: 'd__Bacteria;p__Spirochaetota;c__Spirochaetia;o__Sphaerochaetales;f__Sphaerochaetaceae;g__Sphaerochaeta;s__Sphaerochaeta sp002432715'}

In [28]:
def report_differences():

    unq_gids = set()

    for gid, d_pct_tax in sorted(CHANGES.items()):
        expected_tax = DF_META.loc[gid]['gtdb_taxonomy'].split(';')
        for pct, tax in sorted(d_pct_tax.items()):
            cur_tax = tax.split(';')
            if expected_tax[0:6] != cur_tax[0:6]:
                print(f'{gid} @ {pct}')
                print(f'Expected: {";".join(expected_tax)}')
                print(f'Result  : {";".join(cur_tax)}')
                print()
                unq_gids.add(gid)

    print('Gids that were affected:')
    print('\n'.join(sorted(unq_gids)))

    return

report_differences()

GCA_002170165.1 @ 50
Expected: d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Flavobacteriales;f__BACL11;g__TMED123;s__TMED123 sp002170165
Result  : d__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__MPNO01;f__UBA2964;g__GCA-2724505;s__GCA-2724505 sp002724505

GCA_002401495.1 @ 50
Expected: d__Bacteria;p__Patescibacteria;c__Paceibacteria;o__UBA9983_A;f__UBA2100;g__UBA2100;s__UBA2100 sp002401495
Result  : d__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhodospirillales;f__Thalassospiraceae;g__Thalassospira;s__Thalassospira sp001613795

GCA_014385135.2 @ 1
Expected: d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__Merdisoma;s__Merdisoma sp014290175
Result  : d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Acutalibacteraceae;g__Pseudoruminococcus_A;s__

GCA_017515185.1 @ 30
Expected: d__Bacteria;p__Actinobacteriota;c__Actinomycetia;o__Actinomycetales;f__Bifidobacteriaceae;g__RGIG7107;s__RGIG7107 sp017538895
Result  : d__Bacteria